# HTML5 zu JasperReports Konverter

Dieses Notebook konvertiert HTML5-Code in JasperReport-Sprache mithilfe von Python-Funktionen.

## Struktur:
1. Konstanten eintragen: HTML-Seitengröße und JasperReport-Größe
2. HTML-Code übergeben und speichern
3. Code-Sektionen
4. Diverse Aufrufe und Ausgaben


## 0. Benötigte Bibliotheken installieren


In [ ]:
# Installation der benötigten Bibliotheken
!pip install beautifulsoup4

# Überprüfen der Installation
import sys
print(f"Python Version: {sys.version}")
try:
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")
    print("Alle benötigten Bibliotheken sind installiert.")
except ImportError as e:
    print(f"Fehler beim Importieren: {e}")


## 1. Konstanten eintragen: HTML-Seitengröße und JasperReport-Größe


In [ ]:
# Konstanten für HTML-Seitengröße
HTML_WIDTH = 1210  # Breite in Pixeln
HTML_HEIGHT = 825  # Höhe in Pixeln
HTML_MARGIN_TOP = 0
HTML_MARGIN_RIGHT = 0
HTML_MARGIN_BOTTOM = 0
HTML_MARGIN_LEFT = 0

# Konstanten für JasperReport-Größe
JASPER_PAGE_WIDTH = 595  # A4 Breite in Punkten
JASPER_PAGE_HEIGHT = 842  # A4 Höhe in Punkten
JASPER_MARGIN_TOP = 20
JASPER_MARGIN_RIGHT = 20
JASPER_MARGIN_BOTTOM = 20
JASPER_MARGIN_LEFT = 20

# Skalierungsfaktor berechnen
SCALE_FACTOR_X = (JASPER_PAGE_WIDTH - JASPER_MARGIN_LEFT - JASPER_MARGIN_RIGHT) / HTML_WIDTH
SCALE_FACTOR_Y = (JASPER_PAGE_HEIGHT - JASPER_MARGIN_TOP - JASPER_MARGIN_BOTTOM) / HTML_HEIGHT

print(f"HTML-Größe: {HTML_WIDTH}x{HTML_HEIGHT} Pixel")
print(f"JasperReport-Größe: {JASPER_PAGE_WIDTH}x{JASPER_PAGE_HEIGHT} Punkte")
print(f"Skalierungsfaktor X: {SCALE_FACTOR_X:.4f}")
print(f"Skalierungsfaktor Y: {SCALE_FACTOR_Y:.4f}")


## 2. HTML-Code übergeben und speichern


In [ ]:
def load_html_from_string(html_string):
    """
    Lädt HTML-Code aus einem String.
    
    Args:
        html_string (str): Der HTML-Code als String
        
    Returns:
        str: Der geladene HTML-Code
    """
    return html_string

def load_html_from_file(file_path):
    """
    Lädt HTML-Code aus einer Datei.
    
    Args:
        file_path (str): Pfad zur HTML-Datei
        
    Returns:
        str: Der geladene HTML-Code
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"Fehler beim Laden der Datei: {e}")
        return ""

def save_html_to_file(html_code, file_path):
    """
    Speichert HTML-Code in einer Datei.
    
    Args:
        html_code (str): Der zu speichernde HTML-Code
        file_path (str): Pfad zur Zieldatei
        
    Returns:
        bool: True bei Erfolg, False bei Fehler
    """
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(html_code)
        print(f"HTML-Code erfolgreich in {file_path} gespeichert.")
        return True
    except Exception as e:
        print(f"Fehler beim Speichern der Datei: {e}")
        return False


In [ ]:
# Beispiel-HTML-Code
example_html = """<!DOCTYPE html>
<html>
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
</head>

<body style="margin: 0;">

<div id="p1" style="overflow: hidden; position: relative; background-color: white; width: 1210px; height: 825px;">

    <!-- Begin shared CSS values -->
    <style class="shared-css" type="text/css" >
        .t {
            transform-origin: bottom left;
            z-index: 2;
            position: absolute;
            white-space: pre;
            overflow: visible;
            line-height: 1.5;
        }
        .text-container {
            white-space: pre;
        }
        @supports (-webkit-touch-callout: none) {
            .text-container {
                white-space: normal;
            }
        }
    </style>
    <!-- End shared CSS values -->

    <!-- Begin inline CSS -->
    <style type="text/css" >
        #t1_1{left:18px;bottom:804px;letter-spacing:0.14px;}
        #t2_1{left:128px;bottom:804px;}
        #t3_1{left:165px;bottom:804px;letter-spacing:0.15px;}
    </style>
    <!-- End inline CSS -->

    <!-- Begin text -->
    <div id="t1_1" class="t">Beispieltext 1</div>
    <div id="t2_1" class="t">Beispieltext 2</div>
    <div id="t3_1" class="t">Beispieltext 3</div>
    <!-- End text -->

</div>
</body>
</html>"""

# HTML-Code laden
html_code = load_html_from_string(example_html)
print(f"HTML-Code geladen, Länge: {len(html_code)} Zeichen")


## 3. Code-Sektionen


### 3.1 HTML-Parsing und Analyse


In [ ]:
import re
from bs4 import BeautifulSoup

def parse_html(html_code):
    """
    Parst HTML-Code mit BeautifulSoup.
    
    Args:
        html_code (str): Der zu parsende HTML-Code
        
    Returns:
        BeautifulSoup: Das geparste HTML-Dokument
    """
    try:
        soup = BeautifulSoup(html_code, 'html.parser')
        return soup
    except Exception as e:
        print(f"Fehler beim Parsen des HTML-Codes: {e}")
        return None

def extract_css_styles(soup):
    """
    Extrahiert CSS-Stile aus dem HTML-Dokument.
    
    Args:
        soup (BeautifulSoup): Das geparste HTML-Dokument
        
    Returns:
        dict: Ein Dictionary mit Element-IDs als Schlüssel und Stilen als Werte
    """
    styles = {}
    
    # Inline-Stile aus style-Tags extrahieren
    style_tags = soup.find_all('style')
    for style_tag in style_tags:
        css_content = style_tag.string
        if css_content:
            # Regex zum Extrahieren von Selektoren und ihren Stilen
            style_matches = re.findall(r'#([\w]+)\s*{([^}]+)}', css_content)
            for selector, style_text in style_matches:
                # Stile in ein Dictionary umwandeln
                style_dict = {}
                style_parts = style_text.split(';')
                for part in style_parts:
                    if ':' in part:
                        prop, value = part.split(':', 1)
                        style_dict[prop.strip()] = value.strip()
                styles[selector] = style_dict
    
    return styles

def extract_elements(soup):
    """
    Extrahiert relevante Elemente aus dem HTML-Dokument.
    
    Args:
        soup (BeautifulSoup): Das geparste HTML-Dokument
        
    Returns:
        dict: Ein Dictionary mit Element-Typen als Schlüssel und Listen von Elementen als Werte
    """
    elements = {
        'div': soup.find_all('div'),
        'table': soup.find_all('table'),
        'img': soup.find_all('img'),
        'p': soup.find_all('p'),
        'span': soup.find_all('span')
    }
    
    return elements


### 3.2 Konvertierung von Positionierungen


In [ ]:
def convert_bottom_to_top(height, css_styles):
    """
    Konvertiert bottom-Positionierungen zu top-Positionierungen.
    
    Args:
        height (int): Die Höhe des HTML-Dokuments
        css_styles (dict): Ein Dictionary mit CSS-Stilen
        
    Returns:
        dict: Das aktualisierte CSS-Stile-Dictionary
    """
    updated_styles = {}
    
    for selector, style_dict in css_styles.items():
        updated_style = style_dict.copy()
        
        # Wenn bottom-Eigenschaft vorhanden ist, in top umwandeln
        if 'bottom' in updated_style:
            bottom_value = updated_style['bottom']
            # Extrahiere den numerischen Wert
            match = re.search(r'(\d+\.?\d*)px', bottom_value)
            if match:
                bottom_px = float(match.group(1))
                top_px = height - bottom_px
                updated_style['top'] = f"{round(top_px)}px"
                del updated_style['bottom']
        
        updated_styles[selector] = updated_style
    
    return updated_styles


### 3.3 Konvertierung zu JasperReports XML


In [ ]:
def create_jasper_xml_header(page_width, page_height, margin_top, margin_right, margin_bottom, margin_left):
    """
    Erstellt den XML-Header für ein JasperReports-Dokument.
    
    Args:
        page_width (int): Seitenbreite
        page_height (int): Seitenhöhe
        margin_top (int): Oberer Rand
        margin_right (int): Rechter Rand
        margin_bottom (int): Unterer Rand
        margin_left (int): Linker Rand
        
    Returns:
        str: Der XML-Header
    """
    header = f"""<?xml version="1.0" encoding="UTF-8"?>
<!-- Created with HTML5 to JasperReports Converter -->
<jasperReport xmlns="http://jasperreports.sourceforge.net/jasperreports" 
              xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
              xsi:schemaLocation="http://jasperreports.sourceforge.net/jasperreports http://jasperreports.sourceforge.net/xsd/jasperreport.xsd" 
              name="HTML5_Converted_Report" 
              pageWidth="{page_width}" 
              pageHeight="{page_height}" 
              topMargin="{margin_top}" 
              rightMargin="{margin_right}" 
              bottomMargin="{margin_bottom}" 
              leftMargin="{margin_left}">
    <property name="com.jaspersoft.studio.data.defaultdataadapter" value="One Empty Record"/>
    <queryString>
        <![CDATA[]]>
    </queryString>
    <background>
        <band splitType="Stretch"/>
    </background>
    <title>
        <band height="{page_height - margin_top - margin_bottom}" splitType="Stretch">
"""
    return header

def create_jasper_xml_footer():
    """
    Erstellt den XML-Footer für ein JasperReports-Dokument.
    
    Returns:
        str: Der XML-Footer
    """
    footer = """        </band>
    </title>
</jasperReport>
"""
    return footer

def convert_element_to_jasper(element_id, element_text, style_dict, scale_factor_x, scale_factor_y):
    """
    Konvertiert ein HTML-Element in ein JasperReports-Element.
    
    Args:
        element_id (str): Die ID des Elements
        element_text (str): Der Text des Elements
        style_dict (dict): Die Stile des Elements
        scale_factor_x (float): Skalierungsfaktor für die X-Achse
        scale_factor_y (float): Skalierungsfaktor für die Y-Achse
        
    Returns:
        str: Das JasperReports-XML für das Element
    """
    # Standardwerte
    left = 0
    top = 0
    width = 100
    height = 20
    
    # Positionierung aus Stilen extrahieren
    if 'left' in style_dict:
        match = re.search(r'(\d+\.?\d*)px', style_dict['left'])
        if match:
            left = float(match.group(1)) * scale_factor_x
    
    if 'top' in style_dict:
        match = re.search(r'(\d+\.?\d*)px', style_dict['top'])
        if match:
            top = float(match.group(1)) * scale_factor_y
    
    # JasperReports-Textelement erstellen
    jasper_element = f"""            <staticText>
                <reportElement x="{int(left)}" y="{int(top)}" width="{int(width)}" height="{int(height)}" uuid="{element_id}"/>
                <textElement>
                    <font size="10"/>
                </textElement>
                <text><![CDATA[{element_text}]]></text>
            </staticText>
"""
    
    return jasper_element

def convert_html_to_jasper(html_code, html_height, jasper_page_width, jasper_page_height, 
                          jasper_margin_top, jasper_margin_right, jasper_margin_bottom, jasper_margin_left,
                          scale_factor_x, scale_factor_y):
    """
    Konvertiert HTML-Code in JasperReports-XML.
    
    Args:
        html_code (str): Der zu konvertierende HTML-Code
        html_height (int): Die Höhe des HTML-Dokuments
        jasper_page_width (int): JasperReports-Seitenbreite
        jasper_page_height (int): JasperReports-Seitenhöhe
        jasper_margin_top (int): JasperReports oberer Rand
        jasper_margin_right (int): JasperReports rechter Rand
        jasper_margin_bottom (int): JasperReports unterer Rand
        jasper_margin_left (int): JasperReports linker Rand
        scale_factor_x (float): Skalierungsfaktor für die X-Achse
        scale_factor_y (float): Skalierungsfaktor für die Y-Achse
        
    Returns:
        str: Das JasperReports-XML
    """
    # HTML parsen
    soup = parse_html(html_code)
    if not soup:
        return ""
    
    # CSS-Stile extrahieren
    css_styles = extract_css_styles(soup)
    
    # Bottom zu Top konvertieren
    css_styles = convert_bottom_to_top(html_height, css_styles)
    
    # JasperReports-XML erstellen
    jasper_xml = create_jasper_xml_header(jasper_page_width, jasper_page_height, 
                                         jasper_margin_top, jasper_margin_right, 
                                         jasper_margin_bottom, jasper_margin_left)
    
    # Elemente konvertieren
    for element_type in ['div', 'p', 'span']:
        elements = soup.find_all(element_type)
        for element in elements:
            element_id = element.get('id', '')
            if element_id and element_id in css_styles:
                element_text = element.get_text()
                jasper_xml += convert_element_to_jasper(element_id, element_text, css_styles[element_id], 
                                                      scale_factor_x, scale_factor_y)
    
    jasper_xml += create_jasper_xml_footer()
    
    return jasper_xml


## 4. Diverse Aufrufe und Ausgaben


In [ ]:
# HTML-Code parsen und analysieren
soup = parse_html(html_code)
css_styles = extract_css_styles(soup)
elements = extract_elements(soup)

print(f"Gefundene CSS-Stile: {len(css_styles)}")
print(f"Gefundene Elemente:")
for element_type, element_list in elements.items():
    print(f"  - {element_type}: {len(element_list)}")


In [ ]:
# Bottom zu Top konvertieren
updated_styles = convert_bottom_to_top(HTML_HEIGHT, css_styles)

# Beispiel für konvertierte Stile anzeigen
for selector, style_dict in list(updated_styles.items())[:3]:
    print(f"Selector: #{selector}")
    for prop, value in style_dict.items():
        print(f"  {prop}: {value}")


In [ ]:
# HTML zu JasperReports konvertieren
jasper_xml = convert_html_to_jasper(html_code, HTML_HEIGHT, 
                                   JASPER_PAGE_WIDTH, JASPER_PAGE_HEIGHT,
                                   JASPER_MARGIN_TOP, JASPER_MARGIN_RIGHT,
                                   JASPER_MARGIN_BOTTOM, JASPER_MARGIN_LEFT,
                                   SCALE_FACTOR_X, SCALE_FACTOR_Y)

# Ersten Teil des JasperReports-XML anzeigen
print(jasper_xml[:500] + "...")


In [ ]:
# JasperReports-XML in Datei speichern
output_file = "converted_report.jrxml"
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(jasper_xml)
print(f"JasperReports-XML erfolgreich in {output_file} gespeichert.")


## 5. Zusätzliche Funktionen und Erweiterungen


### 5.1 Tabellen konvertieren


In [ ]:
def convert_table_to_jasper(table_element, scale_factor_x, scale_factor_y):
    """
    Konvertiert eine HTML-Tabelle in ein JasperReports-Tabellenelement.
    
    Args:
        table_element (BeautifulSoup.element): Das HTML-Tabellenelement
        scale_factor_x (float): Skalierungsfaktor für die X-Achse
        scale_factor_y (float): Skalierungsfaktor für die Y-Achse
        
    Returns:
        str: Das JasperReports-XML für die Tabelle
    """
    # Implementierung für Tabellen-Konvertierung
    # (Vereinfachte Version für dieses Beispiel)
    return "<!-- Tabellen-Konvertierung noch nicht implementiert -->"


### 5.2 Bilder konvertieren


In [ ]:
def convert_image_to_jasper(img_element, scale_factor_x, scale_factor_y):
    """
    Konvertiert ein HTML-Bild in ein JasperReports-Bildelement.
    
    Args:
        img_element (BeautifulSoup.element): Das HTML-Bildelement
        scale_factor_x (float): Skalierungsfaktor für die X-Achse
        scale_factor_y (float): Skalierungsfaktor für die Y-Achse
        
    Returns:
        str: Das JasperReports-XML für das Bild
    """
    # Implementierung für Bild-Konvertierung
    # (Vereinfachte Version für dieses Beispiel)
    return "<!-- Bild-Konvertierung noch nicht implementiert -->"


### 5.3 Komplexe Layouts


In [ ]:
def handle_complex_layout(html_code):
    """
    Behandelt komplexe Layouts im HTML-Code.
    
    Args:
        html_code (str): Der HTML-Code mit komplexem Layout
        
    Returns:
        str: JasperReports-XML für das komplexe Layout
    """
    # Implementierung für komplexe Layouts
    # (Vereinfachte Version für dieses Beispiel)
    return "<!-- Komplexe Layout-Konvertierung noch nicht implementiert -->"